In [1]:
from tqdm.auto import tqdm
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data_utils
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
from pathlib import Path
import torchvision.models as models
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Dataset

In [3]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip

--2025-04-19 08:55:21--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.207, 173.194.64.207, 173.194.206.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G   271MB/s    in 12s     

2025-04-19 08:55:33 (296 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



# Data processing

In [4]:


def create_dataloaders(
    dataset_path: str,
    num_classes: int = 10,
    data_augmentation: bool = False,
    batch_size: int = 64,
    val_ratio: float = 0.2,
    num_workers: int = 2,
    seed: int = 42,
    resize: tuple[int, int] = (224, 224), #googlenet
):
    # 1) Normalization constants
    mean = [0.4712, 0.46, 0.389]
    std  = [0.19, 0.18, 0.18]

    # 2) Build a single train transform (conditional augment) + a test transform
    # ----------------------------------------------------------------------------
    train_ops = [
        transforms.Resize(resize),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]
    if data_augmentation:
        # insert augment ops *before* ToTensor
        aug_ops = [
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(30),
        ]
        train_ops[0:0] = aug_ops  # prepend aug ops to the pipeline

    train_transform = transforms.Compose(train_ops)

    test_transform = transforms.Compose([
        transforms.Resize(resize),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])

    # 3) Load the full train & test datasets once
    train_dir = Path(dataset_path) / "train"
    val_dir   = Path(dataset_path) / "val"

    full_train = datasets.ImageFolder(train_dir, transform=train_transform)
    test_ds    = datasets.ImageFolder(val_dir,   transform=test_transform)

    # 4) Stratified split: ensures each class is represented in train & val
    splitter = StratifiedShuffleSplit(
        n_splits=1,
        test_size=val_ratio,
        random_state=seed
    )
    train_idx, val_idx = next(splitter.split(full_train.samples, full_train.targets))

    train_ds = Subset(full_train, train_idx)

    # For val we want the *same* Resize+Normalize but **no** augmentation:
    val_base = datasets.ImageFolder(train_dir, transform=test_transform)
    val_ds   = Subset(val_base, val_idx)

    # 5) DataLoader kwargs for speed
    loader_kwargs = dict(
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
        persistent_workers=True,
    )

    train_loader = DataLoader(train_ds, shuffle=True,  **loader_kwargs)
    val_loader   = DataLoader(val_ds,   shuffle=False, **loader_kwargs)
    test_loader  = DataLoader(test_ds,  shuffle=False, **loader_kwargs)

    # 6) Return loaders + class names
    return train_loader, val_loader, test_loader



# Train & Val

In [5]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss, correct, count = 0.0, 0, 0

    for imgs, labels in tqdm(loader, desc="  ▶ Training", leave=False):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        count   += imgs.size(0)

    avg_loss = total_loss / count
    accuracy = correct / count * 100
    return avg_loss, accuracy


def validate(model, loader, criterion):
    model.eval()
    total_loss, correct, count = 0.0, 0, 0

    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="  ▶ Validating", leave=False):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            count   += imgs.size(0)

    avg_loss = total_loss / count
    accuracy = correct / count * 100
    return avg_loss, accuracy



# Training Model

In [7]:
# Function to train the model
def train_model(model, train_loader, val_loader, loss_func, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, loss_func)

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}')

        val_loss, val_acc =  validate(model, val_loader, loss_func)

        print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}')
        wandb.log({'train_loss': train_loss, 'train_acc': train_acc, 'val_loss' : val_loss, 'val_acc' : val_acc, 'epochs' : epoch + 1})




# Fine tuning

In [8]:
# Fine-tuning function
def fine_tune(model, strategy='freeze_all'):
    if strategy == 'freeze_all':
        # Freeze all layers except the final classification layer
        for name, param in model.named_parameters():
            if "fc" not in name:  # Skip parameters of the final fully connected layer
                param.requires_grad = False
    elif strategy == 'unfreeze_all':
        # Unfreeze all layers and train the entire model
        for param in model.parameters():
            param.requires_grad = True
    elif strategy == 'freeze_first_5':
        # Unfreeze and fine-tune only a subset of layers (e.g., only top layers)
        for i, param in enumerate(model.parameters()):
            if i < 5:
                param.requires_grad = False
    elif strategy == 'freeze_first_15':
        # Unfreeze and fine-tune only a subset of layers (e.g., only top layers)
        for i, param in enumerate(model.parameters()):
            if i < 15:
                param.requires_grad = False

    return model


# Main function

In [10]:
sweep_config = {
    'method': 'grid',
    'name' : 'part-b-googlenet-lr',
    'metric': {
      'name': 'val_acc',
      'goal': 'maximize'
    },
    'parameters': {
        'strategy': {
          'values': ["freeze_all", "unfreeze_all", "freeze_first_5", "freeze_first_15"]
        },
        'learning_rate': {
            'values':[1e-3,1e-4]
        },
    }
}



def main():
    wandb.init(project="Assignment 2")
    config = wandb.config

    run_name = f"googlenet_strat={config.strategy}_lr={config.learning_rate}"
    # Set the run name
    wandb.run.name = run_name
    wandb.run.save()


    # Load pre-trained model
    model = models.googlenet(pretrained=True)

    num_classes = 10  # Number of classes in iNaturalist dataset

    num_features = model.fc.in_features

    model.fc = nn.Linear(num_features, num_classes)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_loader, val_loader, test_loader = create_dataloaders(dataset_path="inaturalist_12K/", data_augmentation=True)

    # Define loss function and optimizer
    loss_func = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    # fine tuning step
    model = fine_tune(model, strategy=config.strategy )
    model.to(device)

    train_model(model, train_loader, val_loader, loss_func, optimizer, num_epochs=7)


sweep_id=wandb.sweep(sweep_config, entity="cs24m033-iit-madras", project="DA6401-A2")
wandb.agent(sweep_id, function= main, count=8)
wandb.finish()

Create sweep with ID: 9w5digx7
Sweep URL: https://wandb.ai/cs24m033-iit-madras/DA6401-A2/sweeps/9w5digx7


wandb: Agent Starting Run: jn7agfq4 with config:
wandb: 	learning_rate: 0.001
wandb: 	strategy: freeze_all


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 192MB/s]


  ▶ Training:   0%|          | 0/125 [00:00<?, ?it/s]

...

  ▶ Training:   0%|          | 0/125 [00:00<?, ?it/s]

Epoch 7/7, Train Loss: 0.2855, Train Accuracy: 91.3989


  ▶ Validating:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 7/7, Val Loss: 0.7001, Val Accuracy: 77.6500


epochs,▁▂▃▅▆▇█
train_acc,▁▄▆▆▇██
train_loss,█▄▃▃▂▁▁
val_acc,▁▅▆█▇▆█
val_loss,█▃▂▁▁▂▁
epochs,7
train_acc,91.39892
train_loss,0.28545
val_acc,77.65
val_loss,0.70011
